# WeatherPy

In [1]:
# Dependencies
import requests
import json
import time
import pandas            as pd
import matplotlib.pyplot as plt
import seaborn           as sns 
from   citipy import citipy

# Generate and or retrieve API key from http://openweathermap.org/appid
# Store Open Weather Map developer API key in config.py then get variable
from config import wkey